# Using dask distributed cluster to speed up computation of PV potential by factor of 10

In [1]:
import xarray as xr
import numpy as np
import time
from dask.distributed import Client
import core as core

import warnings
warnings.filterwarnings("ignore")

In [2]:
# location of era5 data on teachinghub
path="/home/voigta80/LEHRE/msc-intro-comp-met-ex-w2024/data/era5/"

Start dask cluster with 10 processes (workers), each with 5 threads. 

In [3]:
client = Client(n_workers=20, threads_per_worker=5)

In [4]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:42853/status,
Dashboard: http://127.0.0.1:42853/status,Workers: 1
Total threads: 5,Total memory: 753.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38787,Workers: 1
Dashboard: http://127.0.0.1:42853/status,Total threads: 5
Started: Just now,Total memory: 753.83 GiB
Comm: tcp://127.0.0.1:34679,Total threads: 5
Dashboard: http://127.0.0.1:37085/status,Memory: 753.83 GiB
Nanny: tcp://127.0.0.1:35941,


In [ ]:
start_time = time.time()
ds=xr.open_mfdataset(path+"era5-2000-*.nc", engine="netcdf4", chunks={"valid_time":1e5} )
ds["wspd"] = core.windspeed(ds)
pvpot = core.pv_pot(ds).groupby(ds.valid_time.dt.month).mean("valid_time").compute()
end_time = time.time()
execution_time = end_time - start_time

In [ ]:
print(f"Execution time: {execution_time:.5f} seconds")

In [ ]:
client.shutdown()

In [ ]:
pvpot